In [1]:
from pyfinite import ffield
import math
import numpy as np

In [3]:
F = ffield.FField(5)


In [8]:
a = 7
F.ShowPolynomial(a) # show the polynomial representation of a
F

In [2]:
def get_function_to_Zhegalkin_coeffs_matrix_G(n):
    if n in get_function_to_Zhegalkin_coeffs_matrix_G.cache:
        return get_function_to_Zhegalkin_coeffs_matrix_G.cache[n]
    
    G_n_prev = get_function_to_Zhegalkin_coeffs_matrix_G(n - 1)
    result = np.asmatrix(np.vstack([np.hstack([G_n_prev, np.zeros(G_n_prev.shape, dtype=int)]),\
                                    np.hstack([G_n_prev, G_n_prev])]))
    
    get_function_to_Zhegalkin_coeffs_matrix_G.cache[n] = result
    return result

get_function_to_Zhegalkin_coeffs_matrix_G.cache = {0: np.matrix([1], dtype=int)}

In [3]:
def get_positional_functions(permutation):
    n = round(math.log(len(permutation), 2))
    result = []
    for i in range(n):
        shift = n - i - 1
        another_function = [ (permutation[j] >> shift) & 1 for j in range(len(permutation)) ]
        result.append(np.matrix(another_function).T)
    return result

In [40]:
get_function_to_Zhegalkin_coeffs_matrix_G(2)

matrix([[1, 0, 0, 0],
        [1, 1, 0, 0],
        [1, 0, 1, 0],
        [1, 1, 1, 1]])

In [4]:
def get_conjunctions(Zhegalkin_coeffs):
    n = round(math.log(len(Zhegalkin_coeffs), 2))
    included_conjunctions = []
    
    for i in range(1, len(Zhegalkin_coeffs)):
        if Zhegalkin_coeffs[i]:
            variable_vector = []
            included_variables = bin(i)[2:].zfill(n)
            for j in range(len(included_variables)):
                if included_variables[j] != '0':
                    variable_vector.append(j + 1)
            included_conjunctions.append(variable_vector)
    
    included_conjunctions.sort(key=lambda conj: (len(conj), conj))
    return included_conjunctions

In [5]:
def str_Zhegalkin_polynomial(Zhegalkin_coeffs):  
    included_conjunctions = get_conjunctions(Zhegalkin_coeffs)
    
    conjunctions_strings = [conjunction_to_str(conj) for conj in included_conjunctions]
    if Zhegalkin_coeffs[0]:
        conjunctions_strings = ['1'] + conjunctions_strings
    return ' ⊕ '.join(conjunctions_strings)

In [6]:
def conjunction_to_str(conjunction):
    """Конвертирует конъюнкцию, представляемую вектором входящих переменных, в строковую запись.
    [1, 3, 4] -> x1x3x4
    с нижними индексами в Libre Office."""
    
    return ''.join(map(lambda var: "x_{{{}}}".format(var), conjunction))

In [7]:
def zhegalkin_polynomials_from_transform(permutation):
    n = round(math.log(len(permutation), 2))
    
    positional_functions = get_positional_functions(permutation)
    #print(positional_functions)
    positional_functions_Zhegalkin_polynomials =\
        [(get_function_to_Zhegalkin_coeffs_matrix_G(n) * positional_function) % 2 for\
         positional_function in positional_functions]
    #print(positional_functions_Zhegalkin_polynomials)
        
    positional_functions_Zhegalkin_polynomials = map(lambda matrix: matrix.A1,\
                                                     positional_functions_Zhegalkin_polynomials)
    positional_functions_Zhegalkin_polynomials = list(positional_functions_Zhegalkin_polynomials)
    
    Zhegalkin_polynomials_strings = map(str_Zhegalkin_polynomial, positional_functions_Zhegalkin_polynomials)
    Zhegalkin_polynomials_strings = list(Zhegalkin_polynomials_strings)
    #print(Zhegalkin_polynomials_strings)
    
    for i in range(len(Zhegalkin_polynomials_strings)):
        Zhegalkin_polynomials_strings[i] = 'y_{{{}}} = '.format(i + 1) + Zhegalkin_polynomials_strings[i]
    return '; '.join(Zhegalkin_polynomials_strings)

In [8]:
def calculate_degree_of_transform(transform):
    n = round(math.log(len(transform), 2))
    
    positional_functions = get_positional_functions(transform)
    
    positional_functions_Zhegalkin_polynomials =\
    [(get_function_to_Zhegalkin_coeffs_matrix_G(n) * positional_function) % 2 for\
     positional_function in positional_functions]
    
    positional_functions_Zhegalkin_polynomials = map(lambda matrix: matrix.A1,\
                                                     positional_functions_Zhegalkin_polynomials)
    positional_functions_Zhegalkin_polynomials = list(positional_functions_Zhegalkin_polynomials)
    
    Zhegalkin_polynomials_conjunctions = map(get_conjunctions, positional_functions_Zhegalkin_polynomials)
    Zhegalkin_polynomials_conjunctions = list(Zhegalkin_polynomials_conjunctions)
    max_deg = 0
    for lst in Zhegalkin_polynomials_conjunctions:
        for list_item in lst:
            if max_deg < len(list_item):
                max_deg = len(list_item)
    return max_deg   
    

In [46]:
zhegalkin_polynomials_from_transform([3,3,3,2])

'y_{1} = 1; y_{2} = 1 ⊕ x_{1}x_{2}'

In [47]:
calculate_degree_of_transform([3,3,3,2])

2

In [9]:
def mult(elem1, elem2, field):
    return field.Multiply(elem1, elem2)

In [10]:
def powF(elem, power, field):
    if power < 1 or isinstance(power, int) == False :
        raise Exception('power must be 1 or more and integer')
    res = elem
    for i in range(power - 1):
        res = mult(res, elem, field)
    return res
        

In [4]:
def getKloostermanApn(n=3):
    if n % 2 == 0:
        raise Exception('n must be odd')
    F = ffield.FField(n)
    power = 2 ** n - 2
    res_transform = []
    for i in range(2**n):
        res_transform.append(powF(i, power, F))
    return res_transform
    

In [19]:
powF(0,9,ffield.FField(2))

0

In [20]:
getKloostermanApn(3)

[0, 1, 5, 6, 7, 2, 3, 4]

In [11]:
def isPermutation(transform):
    return len(set(transform)) == len(transform)

In [22]:
isPermutation(getKloostermanApn(9))

True

In [23]:
zhegalkin_polynomials_from_transform(getKloostermanApn(3))

'y_{1} = x_{1} ⊕ x_{2} ⊕ x_{1}x_{3}; y_{2} = x_{1} ⊕ x_{2}x_{3}; y_{3} = x_{1} ⊕ x_{2} ⊕ x_{3} ⊕ x_{1}x_{2}'

In [24]:
calculate_degree_of_transform(getKloostermanApn(3))

2

In [53]:
for i in range(3, 13, 2):
    print("n= " + str(i) + " d= " + str(calculate_degree_of_transform(getKloostermanApn(i))))

n= 3 d= 2
n= 5 d= 4
n= 7 d= 6
n= 9 d= 8
n= 11 d= 10


In [28]:
print(math.gcd(1, 19))

1


In [12]:
def get_i_list_gcd_n_1(n):
    res_l = []
    i = 1
    while i <= n:
        if math.gcd(n, i) == 1:
            res_l.append(i)
        i += 1
    return res_l            

In [13]:
for i in range(3, 13):
    get_i_list_gcd_n_1(i)

In [38]:
def getGoldApn(n, i):
    if math.gcd(n,i) != 1:
        raise Exception('gcd(n,i) must be 1')
    F = ffield.FField(n)
    power = 2 ** i + 1
    res_transform = []
    for i in range(2**n):
        res_transform.append(powF(i, power, F))
    return res_transform
    

In [52]:
print("Gold apn x ^ (2 ^ i + 1)")
for n in range(2, 13):
    powers = get_i_list_gcd_n_1(n)
    for i in powers:
        gold_apn = getGoldApn(n, i)
        degree = calculate_degree_of_transform(gold_apn)
        print('n=' + str(n) + ', i=' + str(i) + ", d=" + str(degree))

n=2, i=1, d=2
n=3, i=1, d=2
n=3, i=2, d=2
n=4, i=1, d=2
n=4, i=3, d=2
n=5, i=1, d=2
n=5, i=2, d=2
n=5, i=3, d=2
n=5, i=4, d=2
n=6, i=1, d=2
n=6, i=5, d=2
n=7, i=1, d=2
n=7, i=2, d=2
n=7, i=3, d=2
n=7, i=4, d=2
n=7, i=5, d=2
n=7, i=6, d=2
n=8, i=1, d=2
n=8, i=3, d=2
n=8, i=5, d=2
n=8, i=7, d=2
n=9, i=1, d=2
n=9, i=2, d=2
n=9, i=4, d=2
n=9, i=5, d=2
n=9, i=7, d=2
n=9, i=8, d=2
n=10, i=1, d=2
n=10, i=3, d=2
n=10, i=7, d=2
n=10, i=9, d=2
n=11, i=1, d=2
n=11, i=2, d=2
n=11, i=3, d=2
n=11, i=4, d=2
n=11, i=5, d=2
n=11, i=6, d=2
n=11, i=7, d=2
n=11, i=8, d=2
n=11, i=9, d=2
n=11, i=10, d=2
n=12, i=1, d=2
n=12, i=5, d=2
n=12, i=7, d=2
n=12, i=11, d=2


In [55]:
perm=[1,1,1,1,1,1,1,1]

r = perm
power=len(perm)

matr=[[0 for i in range(len(r))] for i in range(len(r))]


for e in range(0,len(r)):
    for a in range (0,len(r)):
        s=perm[a^e]^perm[a]
        matr[e][s]+=1

print("Division by: "+str(power))
for i in range(0,len(r)):
    print(matr[i])

Division by: 8
[8, 0, 0, 0, 0, 0, 0, 0]
[8, 0, 0, 0, 0, 0, 0, 0]
[8, 0, 0, 0, 0, 0, 0, 0]
[8, 0, 0, 0, 0, 0, 0, 0]
[8, 0, 0, 0, 0, 0, 0, 0]
[8, 0, 0, 0, 0, 0, 0, 0]
[8, 0, 0, 0, 0, 0, 0, 0]
[8, 0, 0, 0, 0, 0, 0, 0]


In [54]:
def getKasamiApn(n, i):
    if math.gcd(n,i) != 1:
        raise Exception('gcd(n,i) must be 1')
    F = ffield.FField(n)
    power = 2 ** (2 * i)- 2 ** i + 1
    res_transform = []
    for i in range(2**n):
        res_transform.append(powF(i, power, F))
    return res_transform

In [58]:
print("Kasami apn 2 ^ 2i - 2 ^ i + 1")
for n in range(2, 11):
    powers = get_i_list_gcd_n_1(n)
    for i in powers:
        kasami_apn = getKasamiApn(n, i)
        degree = calculate_degree_of_transform(kasami_apn)
        print('n=' + str(n) + ', i=' + str(i) + ", d=" + str(degree))

Kasami apn 2 ^ 2i - 2 ^ i + 1
n=2, i=1, d=2
n=3, i=1, d=2
n=3, i=2, d=2
n=4, i=1, d=2
n=4, i=3, d=2
n=5, i=1, d=2
n=5, i=2, d=3
n=5, i=3, d=3
n=5, i=4, d=2
n=6, i=1, d=2
n=6, i=5, d=2
n=7, i=1, d=2
n=7, i=2, d=3
n=7, i=3, d=4
n=7, i=4, d=4
n=7, i=5, d=3
n=7, i=6, d=2
n=8, i=1, d=2
n=8, i=3, d=4
n=8, i=5, d=4
n=8, i=7, d=2
n=9, i=1, d=2
n=9, i=2, d=3
n=9, i=4, d=5
n=9, i=5, d=5
n=9, i=7, d=3
n=9, i=8, d=2
n=10, i=1, d=2
n=10, i=3, d=4
n=10, i=7, d=4


KeyboardInterrupt: 

In [60]:
def getWelchApn(t):
    n = 2 * t + 1
    power = 2 ** t + 3
    F = ffield.FField(n)
    res_transform = []
    for i in range(2**n):
        res_transform.append(powF(i, power, F))
    return res_transform    

In [ ]:
print("Welch apn 2 ^ t + 3 n = 2t+1")
for t in range(2, 13):
    welch_apn = getWelchApn(t)
    degree = calculate_degree_of_transform(welch_apn)
    print('n=' + str(2 * t + 1) + ', t=' + str(t) + ", d=" + str(degree))

Welch apn 2 ^ t + 3 n = 2t+1
n=5, t=2, d=3
n=7, t=3, d=3
n=9, t=4, d=3
n=11, t=5, d=3
n=13, t=6, d=3


In [19]:
def getNihoApn(t):
    n = 2 * t + 1
    power = 'Not a power'
    if t % 2 == 0:
        power = int(2 ** t + 2 ** (t / 2) - 1)
    else:
        power = int(2 ** t + 2 **((3 * t + 1) / 2) - 1)
    F = ffield.FField(n)
    res_transform = []
    for i in range(2**n):
        res_transform.append(powF(i, power, F))
    return res_transform  

In [20]:
print("Noho apn ")
for t in range(2, 7):
    niho_apn = getNihoApn(t)
    degree = calculate_degree_of_transform(niho_apn)
    print('n=' + str(2 * t + 1) + ', t=' + str(t) + ", d=" + str(degree))

Noho apn 
n=5, t=2, d=2
n=7, t=3, d=4
n=9, t=4, d=3
n=11, t=5, d=6
n=13, t=6, d=4


In [18]:
def getInversionApn(t):
    n = 2 * t + 1
    power = 2 ** (2 * t) - 1
    F = ffield.FField(n)
    res_transform = []
    for i in range(2**n):
        res_transform.append(powF(i, power, F))
    return res_transform  

In [19]:
print("Inversion apn")
for t in range(1, 7):
    inversion_apn = getInversionApn(t)
    degree = calculate_degree_of_transform(inversion_apn)
    print('n=' + str(2 * t + 1) + ', t=' + str(t) + ", d=" + str(degree))

Inversion apn
n=3, t=1, d=2
n=5, t=2, d=4
n=7, t=3, d=6
n=9, t=4, d=8
n=11, t=5, d=10
n=13, t=6, d=12


In [20]:
def getDobbertinApn(i):
    n = 5 * i
    power = 2 ** (4 * i) + 2 ** (3 * i) + 2 ** (2 * i) + 2 ** i - 1
    F = ffield.FField(n)
    res_transform = []
    for i in range(2**n):
        res_transform.append(powF(i, power, F))
    return res_transform  

In [23]:
print("Dobbertin apn")
for i in range(1, 3):
    dobbertin_apn = getDobbertinApn(i)
    degree = calculate_degree_of_transform(dobbertin_apn)
    print('n=' + str(5 * i) + ', t=' + str(i) + ", d=" + str(degree))

Dobbertin apn
n=5, t=1, d=4
n=10, t=2, d=5


In [14]:
def trace(elem, power_of_power, n, field):
    res = elem
    #powF(elem, power, field)
    for i in range(1, n):
        res ^= powF(elem, 2 ** (power_of_power * i), field)
    return res
    

In [21]:
F = ffield.FField(6)
print(trace(12, 3, 6, F))

52


In [32]:
def first_ccz_eq_to_power_f(n, i):
    if math.gcd(n,i) != 1:
        raise Exception('gcd(n,i) must be 1')
    if n % 2 != 0:
        raise Exception('n must be even')
    if n < 4:
        raise Exception('n is less than 4')
    F = ffield.FField(n)
    res_transform = []
    for x in range(2**n):
        x_power_2i_1 = powF(x, 2 ** i + 1, F)
        trace_part = trace(x_power_2i_1, 1, n, F)
        r = x_power_2i_1 ^ mult((powF(x, 2 ** i, F) ^ x ^ 1), trace_part, F)
        res_transform.append(r)
    return res_transform
    

In [33]:
first_ccz_eq_to_power_f(4, 1)

[0, 1, 15, 8, 10, 12, 1, 1, 15, 10, 12, 15, 10, 8, 12, 8]

In [49]:
print("first_ccz_eq_to_power_f")
for n in range(4, 14, 2):
    powers = get_i_list_gcd_n_1(n)
    for i in powers:
        first_ccz = first_ccz_eq_to_power_f(n, i)
        degree = calculate_degree_of_transform(first_ccz)
        print('n=' + str(n) + ', i=' + str(i) + ", d=" + str(degree))

first_ccz_eq_to_power_f
n=4, i=1, d=3
n=4, i=3, d=3
n=6, i=1, d=3
n=6, i=5, d=3
n=8, i=1, d=3
n=8, i=3, d=3
n=8, i=5, d=3
n=8, i=7, d=3
n=10, i=1, d=3
n=10, i=3, d=3
n=10, i=7, d=3
n=10, i=9, d=3
n=12, i=1, d=3
n=12, i=5, d=3
n=12, i=7, d=3
n=12, i=11, d=3


In [16]:
def second_ccz_eq_to_power_f(n, i):
    #trace(elem, power_of_power, n, field)
    if math.gcd(n,i) != 1:
        raise Exception('gcd(n,i) must be 1')
    if n % 2 != 0:
        raise Exception('n must be even')
    if n % 3 != 0:
        raise Exception('n % 3 != 0')
    F = ffield.FField(n)
    res_transform = []
    for x in range(2**n):
        for_tr_1 = powF(x, 2 * (2 ** i + 1), F) ^ powF(x, (4 * (2 ** i + 1)) % (2 ** n - 1), F)
        first_part = x ^ trace(for_tr_1, 3, n, F)
        for_tr_2 = powF(x, 2 ** i + 1, F) ^ powF(x, ((2 ** (2 * i)) * (2 ** i + 1)) % (2 ** n - 1), F)
        second_part = mult(trace(x, 1, n, F), trace(for_tr_2, 3, n, F), F)
        r = powF(first_part ^ second_part, 2 ** i + 1, F)
        res_transform.append(r)
    return res_transform


In [17]:
print("second_ccz_eq_to_power_f")
for k in range(2, 14, 2):
    n = 3 * k
    powers = get_i_list_gcd_n_1(n)
    for i in powers:
        second_ccz = second_ccz_eq_to_power_f(n, i)
        degree = calculate_degree_of_transform(second_ccz)
        print('n=' + str(n) + ', i=' + str(i) + ", d=" + str(degree))

second_ccz_eq_to_power_f
n=6, i=1, d=4
n=6, i=5, d=4


KeyboardInterrupt: 

In [22]:
def ordF(elem, F):
    if elem == 0:
        raise Exception('elem is zero')
    resOrd = 1
    g = elem
    while g != 1:
        g = mult(g, elem, F)
        resOrd += 1
    return resOrd

In [23]:
F = ffield.FField(6)
print(ordF(1, F))

1


In [36]:
def first_not_ccz_eq(k, s, w):
    if k < 4:
        raise Exception('k is less then 4')
    if math.gcd(k, 3) != 1:
        raise Exception('gcd(k,3) != 1')
    if math.gcd(s,3 * k) != 1:
        raise Exception('gcd(s,3 * k) != 1')
    n = 3 * k
    i = (s * k) % 3
    m = 3 * i
    F = ffield.FField(n)
    res_transform = []
    for x in range(2 ** n):
        r = powF(x, (2 ** s + 1) % (2 ** n - 1), F) ^ mult(w, powF(x, (2 ** (i * k) + 2 ** (m * k + s)) % (2 ** n - 1),F), F)
        res_transform.append(r)
    return res_transform

In [41]:
def generate_w_with_order(n, order, limit = ''):
    res_w = []
    F = ffield.FField(n)
    cnt = 0
    for x in range(1, 2 ** n):
        if ordF(x,F) == order:
            res_w.append(x)
            if limit != '':
                cnt += 1
                if limit == cnt:
                    break
    return res_w

In [42]:
print("first_not_ccz_eq")
for k in range(4, 5):
    if math.gcd(k, 3) == 1:
        ws = generate_w_with_order(3 * k, 2 ** (2 * k) + 2 ** k + 1, 8)
        print("ws generated!")
        for s in range(1, 3 * k):
            if math.gcd(s, 3 * k) == 1:            
                for w in ws:
                    first_unccz = first_not_ccz_eq(k, s, w)
                    degree = calculate_degree_of_transform(first_unccz)
                    print('n=' + str(3 * k) + ', s=' + str(s) + " w=" + str(w) +", d=" + str(degree))

first_not_ccz_eq
ws generated!
n=12, s=1 w=31, d=2
n=12, s=1 w=107, d=2
n=12, s=1 w=134, d=2
n=12, s=1 w=139, d=2
n=12, s=1 w=161, d=2
n=12, s=1 w=176, d=2
n=12, s=1 w=200, d=2
n=12, s=1 w=201, d=2
n=12, s=5 w=31, d=2
n=12, s=5 w=107, d=2
n=12, s=5 w=134, d=2
n=12, s=5 w=139, d=2
n=12, s=5 w=161, d=2
n=12, s=5 w=176, d=2
n=12, s=5 w=200, d=2
n=12, s=5 w=201, d=2
n=12, s=7 w=31, d=2
n=12, s=7 w=107, d=2
n=12, s=7 w=134, d=2
n=12, s=7 w=139, d=2
n=12, s=7 w=161, d=2
n=12, s=7 w=176, d=2
n=12, s=7 w=200, d=2
n=12, s=7 w=201, d=2
n=12, s=11 w=31, d=2
n=12, s=11 w=107, d=2
n=12, s=11 w=134, d=2


KeyboardInterrupt: 

In [30]:
def second_not_ccz_eq(k, s, w):
    if k < 3:
        raise Exception('k is less then 3')
    if math.gcd(k, 2) != 1:
        raise Exception('gcd(k,2) != 1')
    if math.gcd(s,2 * k) != 1:
        raise Exception('gcd(s,2 * k) != 1')
    n = 4 * k
    i = (s * k) % 4
    m = 4 * i
    F = ffield.FField(n)
    res_transform = []
    for x in range(2 ** n):
        r = powF(x, (2 ** s + 1) % (2 ** n - 1), F) ^ mult(w, powF(x, (2 ** (i * k) + 2 ** (m * k + s)) % (2 ** n - 1),F), F)
        res_transform.append(r)
    return res_transform


In [43]:
print("second_not_ccz_eq")
for k in range(3, 6):
    if math.gcd(k, 2) == 1:
        ws = generate_w_with_order(4 * k, 2 ** (3 * k) + 2 ** (2 * k) + 2 ** k + 1, 5)
        print("ws generated!")
        for s in range(1, 2 * k):
            if math.gcd(s, 2 * k) == 1:            
                for w in ws:
                    second_unccz = second_not_ccz_eq(k, s, w)
                    degree = calculate_degree_of_transform(second_unccz)
                    print('n=' + str(4 * k) + ', s=' + str(s) + " w=" + str(w) +", d=" + str(degree))

second_not_ccz_eq
ws generated!
n=12, s=1 w=10, d=2
n=12, s=1 w=11, d=2
n=12, s=1 w=63, d=2
n=12, s=1 w=68, d=2
n=12, s=1 w=69, d=2
n=12, s=5 w=10, d=2
n=12, s=5 w=11, d=2
n=12, s=5 w=63, d=2
n=12, s=5 w=68, d=2
n=12, s=5 w=69, d=2
ws generated!


KeyboardInterrupt: 

In [44]:
def find_correct_p_for_third_not_ccz(n):
    for p in range(2, n):
        if p != 3 and math.gcd(p, n) == 1:
            return p


In [49]:
def third_not_ccz_eq(n):
    if n < 7:
        raise Exception('n is less then 7')
    if n <= 2 * find_correct_p_for_third_not_ccz(n):
        raise Exception('n  <= 2 * p')
    F = ffield.FField(n)
    res_transform = []
    for x in range(2 ** n):
        pow_3 = powF(x, 3, F) 
        r = pow_3 ^ trace(powF(pow_3, 3, F), 1, n, F)
        res_transform.append(r)
    return res_transform

In [50]:
def generate_valid_n_for_third_not_ccz(upper_limit = 10):
    res_l = []
    for n in range(7, upper_limit):
        if n > 2 * find_correct_p_for_third_not_ccz(n):
            res_l.append(n)
    return res_l        

In [51]:
print("third_not_ccz_eq")
for n in generate_valid_n_for_third_not_ccz(14):
    third_unccz = third_not_ccz_eq(n)
    degree = calculate_degree_of_transform(third_unccz)
    print('n=' + str(n) + ", d=" + str(degree))

third_not_ccz_eq
n=7, d=2
n=9, d=2
n=11, d=2
n=12, d=2
n=13, d=2
